# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 8 2022 11:11AM,243661,21,622918,"NBTY Global, Inc.",Released
1,Aug 8 2022 11:07AM,243659,19,8376238,"Innogenix, LLC",Released
2,Aug 8 2022 11:07AM,243659,19,8376239,"Innogenix, LLC",Released
3,Aug 8 2022 11:07AM,243659,19,8376240,"Innogenix, LLC",Released
4,Aug 8 2022 11:07AM,243659,19,8376241,"Innogenix, LLC",Released
5,Aug 8 2022 11:07AM,243659,19,8376242,"Innogenix, LLC",Released
6,Aug 8 2022 11:07AM,243659,19,8376243,"Innogenix, LLC",Released
7,Aug 8 2022 11:07AM,243659,19,8376244,"Innogenix, LLC",Released
8,Aug 8 2022 11:07AM,243659,19,8376245,"Innogenix, LLC",Released
9,Aug 8 2022 11:07AM,243659,19,8376246,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,243657,Released,1
34,243658,Released,1
35,243659,Released,24
36,243660,Released,1
37,243661,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243657,NaN,1.0
243658,NaN,1.0
243659,NaN,24.0
243660,NaN,1.0
243661,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243601,4.0,1.0
243608,0.0,2.0
243612,1.0,0.0
243613,0.0,3.0
243614,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243601,4,1
243608,0,2
243612,1,0
243613,0,3
243614,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243601,4,1
1,243608,0,2
2,243612,1,0
3,243613,0,3
4,243614,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243601,4,1
1,243608,,2
2,243612,1,
3,243613,,3
4,243614,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 8 2022 11:11AM,243661,21,"NBTY Global, Inc."
1,Aug 8 2022 11:07AM,243659,19,"Innogenix, LLC"
25,Aug 8 2022 11:06AM,243660,10,"Snap Medical Industries, LLC"
26,Aug 8 2022 11:00AM,243655,10,Eywa Pharma Inc.
27,Aug 8 2022 11:00AM,243649,15,"Carwin Pharmaceutical Associates, LLC"
31,Aug 8 2022 11:00AM,243658,21,"NBTY Global, Inc."
32,Aug 8 2022 10:59AM,243657,21,"NBTY Global, Inc."
33,Aug 8 2022 10:55AM,243631,10,ISDIN Corporation
45,Aug 8 2022 10:53AM,243648,15,"Brookfield Pharmaceuticals, LLC"
69,Aug 8 2022 10:52AM,243647,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 8 2022 11:11AM,243661,21,"NBTY Global, Inc.",,1
1,Aug 8 2022 11:07AM,243659,19,"Innogenix, LLC",,24
2,Aug 8 2022 11:06AM,243660,10,"Snap Medical Industries, LLC",,1
3,Aug 8 2022 11:00AM,243655,10,Eywa Pharma Inc.,,1
4,Aug 8 2022 11:00AM,243649,15,"Carwin Pharmaceutical Associates, LLC",,4
5,Aug 8 2022 11:00AM,243658,21,"NBTY Global, Inc.",,1
6,Aug 8 2022 10:59AM,243657,21,"NBTY Global, Inc.",,1
7,Aug 8 2022 10:55AM,243631,10,ISDIN Corporation,,12
8,Aug 8 2022 10:53AM,243648,15,"Brookfield Pharmaceuticals, LLC",,24
9,Aug 8 2022 10:52AM,243647,15,"Alliance Pharma, Inc.",,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 11:11AM,243661,21,"NBTY Global, Inc.",1,
1,Aug 8 2022 11:07AM,243659,19,"Innogenix, LLC",24,
2,Aug 8 2022 11:06AM,243660,10,"Snap Medical Industries, LLC",1,
3,Aug 8 2022 11:00AM,243655,10,Eywa Pharma Inc.,1,
4,Aug 8 2022 11:00AM,243649,15,"Carwin Pharmaceutical Associates, LLC",4,
5,Aug 8 2022 11:00AM,243658,21,"NBTY Global, Inc.",1,
6,Aug 8 2022 10:59AM,243657,21,"NBTY Global, Inc.",1,
7,Aug 8 2022 10:55AM,243631,10,ISDIN Corporation,12,
8,Aug 8 2022 10:53AM,243648,15,"Brookfield Pharmaceuticals, LLC",24,
9,Aug 8 2022 10:52AM,243647,15,"Alliance Pharma, Inc.",7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 11:11AM,243661,21,"NBTY Global, Inc.",1,
1,Aug 8 2022 11:07AM,243659,19,"Innogenix, LLC",24,
2,Aug 8 2022 11:06AM,243660,10,"Snap Medical Industries, LLC",1,
3,Aug 8 2022 11:00AM,243655,10,Eywa Pharma Inc.,1,
4,Aug 8 2022 11:00AM,243649,15,"Carwin Pharmaceutical Associates, LLC",4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 11:11AM,243661,21,"NBTY Global, Inc.",1.0,NaN
1,Aug 8 2022 11:07AM,243659,19,"Innogenix, LLC",24.0,NaN
2,Aug 8 2022 11:06AM,243660,10,"Snap Medical Industries, LLC",1.0,NaN
3,Aug 8 2022 11:00AM,243655,10,Eywa Pharma Inc.,1.0,NaN
4,Aug 8 2022 11:00AM,243649,15,"Carwin Pharmaceutical Associates, LLC",4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 11:11AM,243661,21,"NBTY Global, Inc.",1.0,0.0
1,Aug 8 2022 11:07AM,243659,19,"Innogenix, LLC",24.0,0.0
2,Aug 8 2022 11:06AM,243660,10,"Snap Medical Industries, LLC",1.0,0.0
3,Aug 8 2022 11:00AM,243655,10,Eywa Pharma Inc.,1.0,0.0
4,Aug 8 2022 11:00AM,243649,15,"Carwin Pharmaceutical Associates, LLC",4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5116220,290.0,4.0
12,487283,2.0,42.0
15,730944,35.0,0.0
16,487235,1.0,1.0
19,730909,25.0,1.0
21,974613,4.0,0.0
22,243627,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5116220,290.0,4.0
1,12,487283,2.0,42.0
2,15,730944,35.0,0.0
3,16,487235,1.0,1.0
4,19,730909,25.0,1.0
5,21,974613,4.0,0.0
6,22,243627,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,290.0,4.0
1,12,2.0,42.0
2,15,35.0,0.0
3,16,1.0,1.0
4,19,25.0,1.0
5,21,4.0,0.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,290.0
1,12,Released,2.0
2,15,Released,35.0
3,16,Released,1.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Executing,4.0,42.0,0.0,1.0,1.0,0.0,0.0
Released,290.0,2.0,35.0,1.0,25.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Executing,4.0,42.0,0.0,1.0,1.0,0.0,0.0
1,Released,290.0,2.0,35.0,1.0,25.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Executing,4.0,42.0,0.0,1.0,1.0,0.0,0.0
1,Released,290.0,2.0,35.0,1.0,25.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()